In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

#******bronze prepare table***********#
customers_src=spark.table("workspace.default.customers")
orders_hist_data = [
    (1, 101, 120.0, "2025-12-01"),
    (1, 102,  75.0, "2025-12-02"),
    (1, 107, 130.0, "2025-12-08"),
    (2, 103,  50.0, "2025-12-03"),
    (2, 108,  50.0, "2025-12-10"),
    (4, 104, 200.0, "2025-12-05"),
    (4, 106, 210.0, "2025-12-07")
]
orders_src = (spark.createDataFrame(orders_hist_data, ["CustomerId","OrderId","Amount","OrderDate"])
              .withColumn("OrderDate", F.to_date("OrderDate")))

customers_bronze = (customers_src
    .withColumn("_ingested_at", F.current_timestamp())
    .withColumn("_source", F.lit("demo_customers"))
)

orders_bronze = (orders_src
    .withColumn("_ingested_at", F.current_timestamp())
    .withColumn("_source", F.lit("demo_orders"))
)

customers_bronze.write.format("delta").mode("overwrite").saveAsTable("workspace.default.bronze_customers")
orders_bronze.write.format("delta").mode("overwrite").saveAsTable("workspace.default.bronze_orders")
display(spark.table("workspace.default.bronze_customers"))
display(spark.table("workspace.default.bronze_orders"))

#******bronze prepare table***********#


In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

#******silver table clean\dedupe***********#
bronze_customers = spark.table("workspace.default.bronze_customers")
silver_customers =(bronze_customers
.withColumn("CustomerName", F.trim(F.col("CustomerName")))
.withColumn("Country", F.upper(F.trim(F.col("Country"))))
.withColumn("CustomerId",F.col("CustomerId").cast("long"))
.withColumn("LastUpdatedDate",F.col("LastUpdatedDate").cast("timestamp"))
)
silver_customers.write.format("delta").mode("overwrite").saveAsTable("workspace.default.silver_customers")
display(bronze_customers)
display(spark.table("workspace.default.silver_customers"))

#**Keep latest
bronze_orders = spark.table("workspace.default.bronze_orders")

w=Window.partitionBy("CustomerId","OrderId").orderBy(F.col("_ingested_at").desc())


silver_orders =(bronze_orders
.withColumn("OrderId",F.col("OrderId").cast("long"))
.withColumn("CustomerId",F.col("CustomerId").cast("long"))
.withColumn("Amount",F.col("Amount").cast("double"))
.withColumn("OrderDate",F.col("OrderDate").cast("date"))
.withColumn("rn",F.row_number().over(w))
.filter(F.col("rn")==1)
.drop("rn")
)
silver_orders.write.format("delta").mode("overwrite").saveAsTable("workspace.default.silver_orders")
display(bronze_orders)
display(spark.table("workspace.default.silver_orders"))


In [0]:
#******Gold aggreagates for reporting***********#
import pyspark.sql.functions as F
from pyspark.sql.window import Window

silver_customers = spark.table("workspace.default.silver_customers")
silver_orders = spark.table("workspace.default.silver_orders")

gold_revenue =(silver_orders
.join(silver_customers,on="CustomerId",how="left")
.groupBy("Country")
.agg(
    F.count("*").alias("TotalOrders"),
    F.sum("Amount").alias("TotalRevenue"),
    F.round(F.avg("Amount"),2).alias("AvgOrderAmount")
    )
.orderBy(F.col("TotalRevenue").desc())
)
gold_revenue.write.format("delta").mode("overwrite").saveAsTable("workspace.default.gold_revenue")
display(gold_revenue)


w_latest = Window.partitionBy("CustomerId").orderBy(F.col("OrderDate").desc(), F.col("OrderId").desc())

gold_latest_order_per_customer = (silver_orders
    .withColumn("rn", F.row_number().over(w_latest))
    .filter(F.col("rn") == 1)
    .drop("rn")
    .join(silver_customers.select("CustomerId","CustomerName","Country"), "CustomerId", "left")
    .select("CustomerId","CustomerName","Country","OrderId","Amount","OrderDate")
)

gold_latest_order_per_customer.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("workspace.default.gold_latest_order_per_customer")
display(spark.table("workspace.default.gold_latest_order_per_customer"))

w=Window.partitionBy(F.date_format(F.col("OrderDate"),"yyyy-MM")).orderBy(F.col("OrderDate").desc(), F.col("OrderId").desc())
gold_monthly_revenue=(silver_orders
                      .withColumn("Month",F.date_format(F.col("OrderDate"),"yyyy-MM"))
                      .withColumn("rn", F.row_number().over(w))
                      .filter(F.col("rn") == 1)
                      .drop("rn")
                      .groupBy("Month")
                      .agg(F.sum("Amount").alias("TotalRevenue"))
                      .orderBy(F.col("Month").asc())
                      )

spark.sql("DROP TABLE IF EXISTS workspace.default.gold_monthly_revenue")                    
gold_monthly_revenue.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("workspace.default.gold_monthly_revenue")
display(spark.table("workspace.default.gold_monthly_revenue"))

